In [ ]:
import pyodbc
import pandas as pd
import datetime as dt
import numpy as np
from datetime import timedelta
import csv
#with open([confidential].csv, 'w', newline='') as f:
    #writer = csv.writer(f)



#  Outfile location to be determined.
 #outfile = "[confidential].txt";

#Code to connect to data warehouse 1. driver = type of server. 
# server = sql server's name 
# database = which database on the sql server you want to access.
# trusted connection = yes just means you won't have a security error thrown.

connection = pyodbc.connect(driver = '{SQL SERVER}', 
                            Server = '[confidential]',
                            Database = '[confidential]',
                            trusted_connection = 'yes')

# Was having trouble without department names to narrow query.
# Used SAS to creat table below and pulled it in as csv.

crc_orditem_query = """
select OrderHeaderID as ohid, CustomerID as cid, oi.SaleDate as sd, Quantity as quant, NetSalesAmount as nsa, 
DiscountAmount as da, dp.SubCategoryCode as subcat, dp.SubCategoryName as subcatname
from fact_OrderItem as oi
inner join dim_Product as dp on oi.ProductID = dp.ProductID
where dp.subcategorycode in ('02','01','03')
and dp.DepartmentCode in ('02')
and dp.CategoryCode = ('04')
and oi.SaleDate >= '2018-06-28' and oi.SaleDate <= '2019-06-28';"""

crc_orderitem = pd.read_sql_query(crc_orditem_query, connection)
print(crc_orderitem.head(100))


crc_orderitem['sd'] = pd.to_datetime(crc_orderitem['sd'])
crc_orderitem['cid'] = crc_orderitem['cid'].fillna(value = -1)
crc_orderitem['cid'] = crc_orderitem['cid'].astype('int64')
crc_orderitem['quant'] = crc_orderitem['quant'].astype('int64')
crc_orderitem['subcat'] = crc_orderitem['subcat'].fillna(value = -1)
crc_orderitem['subcat'] = crc_orderitem['subcat'].astype('int64')

def convert_currency(val):
    """ \
    Convert the string number value to a float
     - Remove $
     - Remove commas
     - Convert to float type
    """
    new_val = val.replace(',','').replace('$', '')
    return float(new_val)

crc_orderitem['nsa'] = crc_orderitem['nsa'].apply(convert_currency)
crc_orderitem['da'] = crc_orderitem['da'].apply(convert_currency)
print(crc_orderitem.dtypes)


In [ ]:
dim_date_query = """
select FY_No, FW_No, icDisplayDate as date
from dim_Date
where icDisplayDate >= '2018-06-28' and icDisplayDate <= '2019-06-28' """
dim_date = pd.read_sql_query(dim_date_query, connection)
dim_date['date'] = pd.to_datetime(dim_date['date'])

crc_orders = pd.merge(crc_orderitem, dim_date, how = 'inner', left_on = 'sd', right_on = 'date')


crc_orders['cake_accesories_fl'] = np.where(crc_orders['subcat'] == 54, 1, 0)
crc_orders['cake_machine_fl'] = np.where(crc_orders['subcat'] == 52, 1, 0)
crc_orders['tools_mats_and_accessories_fl'] = np.where(crc_orders['subcat'] == 2, 1, 0)
crc_orders['machines_bundles_fl'] = np.where(crc_orders['subcat'] == 1, 1, 0)
crc_orders['vinyls_and_iron-on_rolls_fl'] = np.where(crc_orders['subcat'] == 3, 1, 0)

#crc_orders.to_csv('[confidential]\\crc_orders.csv')

crc_orders.dtypes

def return_inst(series):
    return series.iloc[0]

# a basic groupby customer id order header id and product subcategory name 
# with dictionary mapping for columns and aggregating functions     
trans_crc = crc_orders.groupby(['cid','ohid','subcatname']).agg({'quant':'sum', 'nsa':'sum', 'da':'sum', 'tools_mats_and_accessories_fl':'max', 'machines_bundles_fl':'max', 'crc Vinyls & Iron-On Rolls_FL':'max', 'sd':'return_inst', 'FY_No':'return_inst', 'FW_No':'return_inst'})


trans_crc = trans_crc.reset_index()

def aur(df):
    return df['nsa'].sum() / df['quant'].sum()

def metrics(df):
    s = []
    trans_ct = df['ohid'].nunique()
    s.append(trans_ct)
    quant_sum = df['quant'].sum()
    s.append(quant_sum)
    quant_mean = df['quant'].sum() / df['ohid'].nunique()
    s.append(quant_mean)
    sales_per_trans = df['nsa'].sum() / df['ohid'].nunique()
    s.append(sales_per_trans)
    sales_per_quant = df['nsa'].sum() / quant_sum
    s.append(sales_per_quant)
    
    return pd.Series(s, index = ['Transactions', 'Total Quantity', 'Quantity per Transaction', \
                                 'Sales per Transaction', 'Sales per Unit'])
 
    
    

trans_FWoI = trans_crc.query('20 <= FW_No <= 26')
fw_subcat_breakout = trans_FWoI.groupby(['FW_No', 'subcatname']).apply(metrics)
fw_overall = trans_FWoI.groupby('FW_No').apply(metrics)
#print(test)
trans_FWoI = trans_crc.query('20 <= FW_No <= 26')
AUR = trans_FWoI.groupby(['FW_No', 'subcatname'], as_index = False).apply(aur)



In [ ]:
trans_crc['cid'] = trans_crc['cid'].astype('int64')
print(trans_crc.dtypes)

# a function returns a series of days between visits to sum up those days eventually
def dbtwv_calc(df):
    if len(df['sd']) == 1:
        df.at[:,'sd'] = -1
        return df['sd']
    
    else:
        df['sd'] = df['sd'].sort_values()
        diff = df['sd'].diff()
        diff = diff.dropna()
        diff = diff.apply(lambda x: x.days)
        return diff

def ohid_ret(s):
    return s
test = trans_crc.loc[trans_crc['cid'] > -1]
trans_freq = test.groupby(['cid','subcatname','sd'], as_index = False).agg({'ohid':ohid_ret})
trans_freq.head(10)


In [ ]:
#trans_freq = trans_freq.reset_index()
print(trans_freq.columns)
test2 = trans_freq.copy()
trans_freq1 = test2.groupby(['cid','subcatname']).apply(dbtwv_calc)
trans_freq1 = trans_freq1.reset_index()
#trans_freq1.to_csv('[confidential]\\test.csv')
trans_freq2 = pd.DataFrame(trans_freq1)
trans_freq2 = trans_freq2.drop('level_2', axis = 1)
trans_freq2 = trans_freq2[trans_freq2.sd != -1]
trans_freq2.query('sd == 0.0')



In [ ]:
dbtwv_lab = ["{0} - {1}".format(days, days + 14) for days in range(15, 226, 15)]
dbtwv_b = [x - 1 for x in range(0,226,15)]
dbtwv_b.append(370)
dbtwv_b.insert(1,6)
dbtwv_b.insert(1,2)
dbtwv_lab.append('225+')
dbtwv_lab[0] = '0-2'
dbtwv_lab.insert(1,'7-14')
dbtwv_lab.insert(1,'3-6')
print(trans_freq1.agg({'sd':'max'}))

trans_freq2['dbtwv_bins'] = pd.cut(x = trans_freq2['sd'], bins = dbtwv_b, labels = dbtwv_lab)
trans_freq2.to_excel('[confidential]\\dbtwv.xlsx')

piv_bysubcat = pd.pivot_table(trans_freq2, index = 'dbtwv_bins', columns = 'subcatname', \
                      values = 'sd', aggfunc = 'count', dropna = False)


piv_bysubcatpct = piv_bysubcat.copy()
for x in piv_bysubcatpct.columns.to_list():
    piv_bysubcatpct[x] = piv_bysubcatpct[x] / piv_bysubcatpct[x].sum()





piv_ov = pd.pivot_table(trans_freq2, index = 'dbtwv_bins', \
                      values = 'sd', \
                         aggfunc = 'count', dropna = False)

piv_ovpct = piv_ov.copy()

for x in piv_ov.columns.to_list():
    piv_ovpct[x] = piv_ovpct[x] / piv_ovpct[x].sum()



def stats(df):
    count = df['sd'].count()
    mean = df['sd'].mean()
    median = df['sd'].median()
    stats = [count, mean, median]
    s = pd.Series(stats)
    return s

stat_bysubcat = trans_freq2.groupby('subcatname', as_index = False).apply(stats)
stat_bysubcat = stat_bysubcat.T
stat_bysubcat = stat_bysubcat.reset_index()
stats_ov = trans_freq2.agg({'sd':['count', 'mean', 'median']})
stats_ov = stats_ov.reset_index()



trans_out = piv_ov.join(piv_bysubcat)
trans_outpct = piv_ovpct.join(piv_bysubcatpct)

print(trans_out)
trans_outpct.to_excel('[confidential]\\dbtwv_dist.xlsx')
stats = pd.merge(stats_ov, stat_bysubcat, left_index = True, right_index = True)
stats = stats.reset_index()
stats.to_csv('[confidential]\\dbtwv_stats.csv')

print("Done!")


In [ ]:
# making some pivot tables for visualizing sales and transaction
# distributions by category and subcategory
trans_crc2 = crc_orders.groupby(['cid','ohid'], as_index = False).agg('tools_mats_and_accessories_fl':'max', 'machines_bundles_fl':'max', 'vinyls_and_iron-on_rolls_fl':'max'})


trans_crc3 = trans_crc2.groupby('cid', as_index = False).agg({'tools_mats_and_accessories_fl':'sum', \
                                                  'machines_bundles_fl':'sum', \
                                       'vinyls_and_iron-on_rolls_fl':'sum'})

trans_crc_cidpres = trans_crc3[trans_crc3['cid'] != -1]

trans_crc_cidpres1 = trans_crc2[trans_crc2['cid'] != -1]

#print(trans_crc_cidpres)

bin_lab = ["{0}".format(trans) for trans in range(1, 10, 1)]
transb = list([x for x in range(0,10,1)])
bin_lab.append('10+')
transb.append(52)


trans_tool1 = trans_crc_cidpres.loc[trans_crc_cidpres['tools_mats_and_accessories_fl'] > 0]
trans_mach1 = trans_crc_cidpres.loc[trans_crc_cidpres['machines_bundles_fl'] > 0]
trans_vinyl1 = trans_crc_cidpres.loc[trans_crc_cidpres['vinyls_and_iron-on_rolls_fl'] > 0]

trans_tool = trans_tool1.loc[:,['cid', 'tools_mats_and_accessories_fl']]
trans_mach = trans_mach1.loc[:,['cid','machines_bundles_fl']]
trans_vinyl = trans_vinyl1.loc[:,['cid', 'vinyls_and_iron-on_rolls_fl']]


trans_tool['Transactions'] = pd.cut(x = trans_tool['tools_mats_and_accessories_fl'], \
                                  bins = transb, labels = bin_lab)
trans_mach['Transactions'] = pd.cut(x = trans_mach['machines_bundles_fl'], \
                                  bins = transb, labels = bin_lab)
trans_vinyl['Transactions'] = pd.cut(x = trans_vinyl['vinyls_and_iron-on_rolls_fl'], \
                                  bins = transb, labels = bin_lab)

#print(trans_tool.head(), trans_vinyl.head(10), trans_mach.head(10))

piv_tool = pd.pivot_table(trans_tool, index = 'Transactions', \
                      values = 'tools_mats_and_accessories_fl', \
                          aggfunc = 'count', dropna = False)
tool_stats = trans_tool.agg({'tools_mats_and_accessories_fl':['count','mean','median']})

piv_mach = pd.pivot_table(trans_mach, index = 'Transactions', \
                      values = 'machines_bundles_fl', \
                          aggfunc = 'count', dropna = False)

mach_stats = trans_mach.agg({'machines_bundles_fl':['count','mean','median']})

piv_vinyl = pd.pivot_table(trans_vinyl, index = 'Transactions', \
                      values = 'vinyls_and_iron-on_rolls_fl', \
                          aggfunc = 'count', dropna = False)
vinyl_stats = trans_vinyl.agg({'vinyls_and_iron-on_rolls_fl':['count','mean','median']})

trans_bysubcat = piv_tool.join([piv_mach, piv_vinyl])

def tot_trans(df):
    tool = df['tools_mats_and_accessories_fl'].sum()
    mach = df['machines_bundles_fl'].sum()
    vinyl = df['vinyls_and_iron-on_rolls_fl'].sum()
    tot  = tool + mach + vinyl
    return tot

trans_ov = trans_crc_cidpres1.groupby('cid').agg({'ohid':'nunique'})
trans_ov = trans_ov.reset_index()
#trans_ov = trans_ov.rename({'cid':'cid', 0:'trans'}, axis = 1)
trans_ov['Transactions'] = pd.cut(x = trans_ov['ohid'], \
                                  bins = transb, labels = bin_lab)
piv_ov = pd.pivot_table(trans_ov, index = 'Transactions', \
                      values = 'ohid', \
                          aggfunc = 'count', dropna = False)

ov_stats = trans_ov.agg({'ohid':['count','mean','median']})

trans_out = piv_ov.join(trans_bysubcat)
trans_outpct = trans_out.copy()


for x in trans_outpct.columns.to_list():
    trans_outpct[x] = trans_outpct[x] / trans_outpct[x].sum()
    

trans_stats_out = ov_stats.join([tool_stats,mach_stats,vinyl_stats])
    
    
print(trans_out, trans_stats_out)

trans_out.to_excel('[confidential]_trans_dist.xlsx')
trans_outpct.to_excel('[confidential]_trans_distpct.xlsx')
trans_stats_out.to_csv('[confidential]_trans_stats.csv')






